# Part 2
# Extra Features for a boost in Model Perfomance 

By GROUP4: Stijn Uijen, Leon van Veldhuijzen and Max van Splunteren.


The code in this file belongs to the second part of the group assignment. Here, we will try to boost 
performance for each of the classification models by adding extra features we think could be relevant to the county data.

In the first part, the extra features are added, in the second part the code is the same as in part one for each of the classification problems, and in the last part the models get retrained with the new data.

#### So the only new code is the 'Add the new features to the county_data' and 'New Models' part.
 

#### Acknowledgements
Parts of the code here were inspired by lectures and materials from Dr. Stevan Rudinac and Rens Dimmendaal
and the book 'Introduction to Machine Learning with Python' by A. C. Muller and S. Guido. 

# Import and Prepare Data

In [1]:
import pandas as pd

county = 'county_facts.csv'

results = 'primary_results.csv'

county_data = pd.read_csv(county)

results_data = pd.read_csv(results)


##### extra features data-sets
extra_county_data = pd.read_excel('international_migrants.xlsx')
edu_data = pd.read_excel('education_counties.xlsx')
unem_data = pd.read_excel('county_unemployment.xlsx')

In [2]:
### filter for results for Democrats and Republicans 
republican_results = results_data[results_data['party'].isin(['Republican'])]

democrat_results = results_data[results_data['party'].isin(['Democrat'])]


In [3]:
# drop all 'whole' state rows and the USA row by checking if 'state_abbreviation' == NaN

# first make a list with indexes of the rows that have to be deleted 
indexes = []

index = 0 

for index, row in county_data.iterrows():
    
    if pd.isnull(row['state_abbreviation']):
        
        indexes.append(index)
        
    index += 1

# now drop these rows by index
county_data = county_data.drop(county_data.index[indexes])

# Add the new features to the county_data

In [4]:
# Now add the new features to the democratic data-set

mig_rate_2011 = pd.DataFrame(data = {'R_INTERNATIONAL_MIG_2011' : [0] * len(county_data['fips'])})
mig_rate_2012 = pd.DataFrame(data = {'R_INTERNATIONAL_MIG_2012' : [0] * len(county_data['fips'])})
mig_rate_2013 = pd.DataFrame(data = {'R_INTERNATIONAL_MIG_2013' : [0] * len(county_data['fips'])})
mig_rate_2014 = pd.DataFrame(data = {'R_INTERNATIONAL_MIG_2014' : [0] * len(county_data['fips'])})
mig_rate_2015 = pd.DataFrame(data = {'R_INTERNATIONAL_MIG_2015' : [0] * len(county_data['fips'])})
mig_rate_2016 = pd.DataFrame(data = {'R_INTERNATIONAL_MIG_2016' : [0] * len(county_data['fips'])})

less_than_highschool_2011_2015 = pd.DataFrame(data = {'less_than_highschool_2011_2015' : [0] * len(county_data['fips'])})
#only_highschool_2011_2015 = pd.DataFrame(data = {'only_highschool_2011_2015' : [0] * len(county_data['fips'])})
# college_degree_2011_2015 = pd.DataFrame(data = {'college_degree_2011_2015' : [0] * len(county_data['fips'])})
# bachelor_or_higher_2011_2015 = pd.DataFrame(data = {'bachelor_or_higher_2011_2015' : [0] * len(county_data['fips'])})

Unemployment_rate_2016 = pd.DataFrame(data = {'Unemployment_rate_2016' : [0] * len(county_data['fips'])})
Median_Household_Income_2015 = pd.DataFrame(data = {'Median_Household_Income_2015' : [0] * len(county_data['fips'])})
Med_HH_Income_Percent_of_State_Total_2015 = \
                pd.DataFrame(data = {'Med_HH_Income_Percent_of_State_Total_2015' : [0] * len(county_data['fips'])})


data_plus = pd.concat([county_data ,mig_rate_2011, mig_rate_2012, mig_rate_2013, 
                             mig_rate_2014, mig_rate_2015, mig_rate_2016,
                             less_than_highschool_2011_2015, 
                             Unemployment_rate_2016, Median_Household_Income_2015,
                             Med_HH_Income_Percent_of_State_Total_2015], axis = 1)


In [5]:
# create dictionaries for the extra variables where the counties are keys and the corresponing value 
# from the feature as the value.

int_mig_dict = {}
int_mig_dict_fips_missing = {}

edu_dict = {}
edu_dict_fips_missing = {}

unem_dict = {}
unem_dict_fips_missing = {}

for index, row in extra_county_data.iterrows():
    
        int_mig_dict[row['FIPS']] = {'R_INTERNATIONAL_MIG_2011': extra_county_data['R_INTERNATIONAL_MIG_2011'][index],
                                 'R_INTERNATIONAL_MIG_2012': extra_county_data['R_INTERNATIONAL_MIG_2012'][index],
                                 'R_INTERNATIONAL_MIG_2013': extra_county_data['R_INTERNATIONAL_MIG_2013'][index],
                                 'R_INTERNATIONAL_MIG_2014': extra_county_data['R_INTERNATIONAL_MIG_2014'][index],
                                 'R_INTERNATIONAL_MIG_2015': extra_county_data['R_INTERNATIONAL_MIG_2015'][index], 
                                 'R_INTERNATIONAL_MIG_2016': extra_county_data['R_INTERNATIONAL_MIG_2016'][index]}
        
        int_mig_dict_fips_missing[row['Area_Name']] = \
                            {'R_INTERNATIONAL_MIG_2011': extra_county_data['R_INTERNATIONAL_MIG_2011'][index],
                                 'R_INTERNATIONAL_MIG_2012': extra_county_data['R_INTERNATIONAL_MIG_2012'][index],
                                 'R_INTERNATIONAL_MIG_2013': extra_county_data['R_INTERNATIONAL_MIG_2013'][index],
                                 'R_INTERNATIONAL_MIG_2014': extra_county_data['R_INTERNATIONAL_MIG_2014'][index],
                                 'R_INTERNATIONAL_MIG_2015': extra_county_data['R_INTERNATIONAL_MIG_2015'][index], 
                                 'R_INTERNATIONAL_MIG_2016': extra_county_data['R_INTERNATIONAL_MIG_2016'][index]}

        edu_dict[row['FIPS']] = {'less_than_highschool_2011_2015':\
                                 edu_data['Percent of adults with less than a high school diploma, 2011-2015'][index]}
                                 
        edu_dict_fips_missing[row['Area_Name']] = {'less_than_highschool_2011_2015': \
                                edu_data['Percent of adults with less than a high school diploma, 2011-2015'][index]}
        
        unem_dict[row['FIPS']] = {'Unemployment_rate_2016': unem_data['Unemployment_rate_2016'][index],
                                 'Median_Household_Income_2015': unem_data['Median_Household_Income_2015'][index],
                                 'Med_HH_Income_Percent_of_State_Total_2015': \
                                  unem_data["Med_HH_Income_Percent_of_State_Total_2015"][index]}
                                 
        unem_dict_fips_missing[row['Area_Name']] = {'Unemployment_rate_2016': unem_data['Unemployment_rate_2016'][index],
                                 'Median_Household_Income_2015': unem_data['Median_Household_Income_2015'][index],
                                 'Med_HH_Income_Percent_of_State_Total_2015': \
                                                    unem_data["Med_HH_Income_Percent_of_State_Total_2015"][index]}

In [6]:
# fill values for migrant features 
for index, row in data_plus.iterrows():
    
    if pd.isnull(row['fips']) and row['area_name'] in int_mig_dict_fips_missing: 
        
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2011'] = \
        int_mig_dict_fips_missing[row['area_name']]['R_INTERNATIONAL_MIG_2011']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2012'] = \
        int_mig_dict_fips_missing[row['area_name']]['R_INTERNATIONAL_MIG_2012']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2013'] = \
        int_mig_dict_fips_missing[row['area_name']]['R_INTERNATIONAL_MIG_2013']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2014'] = \
        int_mig_dict_fips_missing[row['area_name']]['R_INTERNATIONAL_MIG_2014']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2015'] = \
        int_mig_dict_fips_missing[row['area_name']]['R_INTERNATIONAL_MIG_2015']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2016'] = \
        int_mig_dict_fips_missing[row['area_name']]['R_INTERNATIONAL_MIG_2016']
        
    elif row['fips'] in int_mig_dict:
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2011'] = int_mig_dict[row['fips']]['R_INTERNATIONAL_MIG_2011']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2012'] = int_mig_dict[row['fips']]['R_INTERNATIONAL_MIG_2012']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2013'] = int_mig_dict[row['fips']]['R_INTERNATIONAL_MIG_2013']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2014'] = int_mig_dict[row['fips']]['R_INTERNATIONAL_MIG_2014']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2015'] = int_mig_dict[row['fips']]['R_INTERNATIONAL_MIG_2015']
        data_plus.loc[index, 'R_INTERNATIONAL_MIG_2016'] = int_mig_dict[row['fips']]['R_INTERNATIONAL_MIG_2016']
        
# fill values for education features 
for index, row in data_plus.iterrows():
    
    if pd.isnull(row['fips']) and row['area_name'] in edu_dict_fips_missing: 
        
        data_plus.loc[index, 'less_than_highschool_2011_2015'] = \
        edu_dict_fips_missing[row['area_name']]['less_than_highschool_2011_2015']

        
    elif row['fips'] in edu_dict:
        data_plus.loc[index, 'less_than_highschool_2011_2015'] = \
        edu_dict[row['fips']]['less_than_highschool_2011_2015']

# fill values for unemployment features 
for index, row in data_plus.iterrows():
    
    if pd.isnull(row['fips']) and row['area_name'] in unem_dict_fips_missing: 
        
        data_plus.loc[index, 'Unemployment_rate_2016'] = \
        unem_dict_fips_missing[row['area_name']]['Unemployment_rate_2016']
        data_plus.loc[index, 'Median_Household_Income_2015'] = \
        unem_dict_fips_missing[row['area_name']]['Median_Household_Income_2015']
        data_plus.loc[index, 'Med_HH_Income_Percent_of_State_Total_2015'] = \
        unem_dict_fips_missing[row['area_name']]['Med_HH_Income_Percent_of_State_Total_2015']
        
        
    elif row['fips'] in unem_dict:
        data_plus.loc[index, 'Unemployment_rate_2016'] = unem_dict[row['fips']]['Unemployment_rate_2016']
        data_plus.loc[index, 'Median_Household_Income_2015'] = \
            unem_dict[row['fips']]['Median_Household_Income_2015']
        data_plus.loc[index, 'Med_HH_Income_Percent_of_State_Total_2015'] = \
            unem_dict[row['fips']]['Med_HH_Income_Percent_of_State_Total_2015']

In [7]:
# drop rows with missing values
county_data = data_plus.dropna(axis=0, how = 'any')

In [8]:
# rename column names
county_data.rename(index=str, columns={
'PST045214':"Population: 2014 estimate",
'PST040210':"Population: 2010 (April 1) estimates base",
'PST120214':"Population: percent change - April 1: 2010 to July 1: 2014",
'POP010210':"Population: 2010",
'AGE135214':"Persons under 5 years: percent: 2014",
'AGE295214':"Persons under 18 years: percent: 2014",
'AGE775214':"Persons 65 years and over: percent: 2014",
'SEX255214':"Female persons: percent: 2014",
'RHI125214':"White alone: percent: 2014",
'RHI225214':"Black or African American alone: percent: 2014",
'RHI325214':"American Indian and Alaska Native alone: percent: 2014",
'RHI425214':"Asian alone: percent: 2014",
'RHI525214':"Native Hawaiian and Other Pacific Islander alone: percent: 2014",
'RHI625214':"Two or More Races: percent: 2014",
'RHI725214':"Hispanic or Latino: percent: 2014",
'HI825214':"White alone: not Hispanic or Latino: percent: 2014",
'POP715213':"Living in same house 1 year & over: percent: 2009-2013",
'POP645213':"Foreign born persons: percent: 2009-2013",
'POP815213':"Language other than English spoken at home: pct age 5+: 2009-2013",
'EDU635213':"High school graduate or higher: percent of persons age 25+: 2009-2013",
'EDU685213':"Bachelor's degree or higher: percent of persons age 25+: 2009-2013",
'VET605213':"Veterans: 2009-2013",
'LFE305213':"Mean travel time to work (minutes): workers age 16+: 2009-2013",
'HSG010214':"Housing units: 2014",
'HSG445213':"Homeownership rate: 2009-2013",
'HSG096213':"Housing units in multi-unit structures: percent: 2009-2013",
'HSG495213':"Median value of owner-occupied housing units: 2009-2013",
'HSD410213':"Households: 2009-2013",
'HSD310213':"Persons per household: 2009-2013",
'INC910213':"Per capita money income in past 12 months (2013 dollars): 2009-2013",
'INC110213':"Median household income: 2009-2013",
'PVY020213':"Persons below poverty level: percent: 2009-2013",
'BZA010213':"Private nonfarm establishments: 2013",
'BZA110213':"Private nonfarm employment:  2013",
'BZA115213':"Private nonfarm employment: percent change: 2012-2013",
'NES010213':"Nonemployer establishments: 2013",
'SBO001207':"Total number of firms: 2007",
'SBO315207':"Black-owned firms: percent: 2007",
'SBO115207':"American Indian- and Alaska Native-owned firms: percent: 2007",
'SBO215207':"Asian-owned firms: percent: 2007",
'SBO515207':"Native Hawaiian- and Other Pacific Islander-owned firms: percent: 2007",
'SBO415207':"Hispanic-owned firms: percent: 2007",
'SBO015207':"Women-owned firms: percent: 2007",
'MAN450207':"Manufacturers shipments: 2007 ($1:000)",
'WTN220207':"Merchant wholesaler sales: 2007 ($1:000)",
'RTN130207':"Retail sales: 2007 ($1:000)",
'RTN131207':"Retail sales per capita: 2007",
'AFN120207':"Accommodation and food services sales: 2007 ($1:000)",
'BPS030214':"Building permits: 2014",
'LND110210':"Land area in square miles: 2010",
'POP060210':"Population per square mile: 2010"}, inplace=True)

C:\development\anaconda\lib\site-packages\pandas\core\frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


# Democrats train/ test split:

In [9]:
# Create Dictionary with fips as keys and winner as value. Use to later acces the winner per row of the county data.
# (fips are the unique county codes)
dem_winners = {}
dem_winners_fips_missing = {}

for index, row in democrat_results.iterrows():
    
    # some county fips are missing, add to seperate dict to link them to the winner
    if pd.isnull(row['fips']) and row['candidate'] == 'Hillary Clinton':
        
        dem_winners_fips_missing[row['county']] = [row['state_abbreviation'], row['fraction_votes'],
                                                   'Hillary Clinton']
        
    elif row['candidate'] == 'Hillary Clinton':
        
        dem_winners[row['fips']] = ['Hillary Clinton', row['fraction_votes']]
                
for index, row in democrat_results.iterrows():
    
    if pd.isnull(row['fips']) and row['candidate'] == 'Bernie Sanders':
        
        if row['fraction_votes'] > dem_winners_fips_missing[row['county']][1]:
             
                dem_winners_fips_missing[row['county']] = [row['state_abbreviation'],
                                                        row['fraction_votes'], 'Bernie Sander']
        
        elif row['fraction_votes'] == dem_winners_fips_missing[row['county']][1]:
             
                dem_winners_fips_missing[row['county']] = [row['state_abbreviation'],'Tie']
    
    
    elif row['candidate'] == 'Bernie Sanders' and row['fraction_votes'] > dem_winners[row['fips']][1]:
        
        dem_winners[row['fips']][0] = 'Bernie Sanders'
        
    
    elif row['candidate'] == 'Bernie Sanders' and row['fraction_votes'] == dem_winners[row['fips']][1]:
        
        dem_winners[row['fips']][0] = 'Tie'


# create the y column: 'democratic_winner'. Set values to 0 for now. Merge with the county data.        
democratic_winner = {'democratic_winner' : [0] * len(county_data['fips'])}

democratic_winner_df = pd.DataFrame(data = democratic_winner)

democratic_data = pd.concat([county_data ,democratic_winner_df], axis = 1)


# Now set the values of the 'democratic_winner' column to the either Bernie, Hillary or Tie. Value stays 0 if 
# the county code is not in the result data-set, and therefore missing
for index, row in democratic_data.iterrows():
    
    # this is the if statement for where country fip is missing and berny is thus the winner. 
    if pd.isnull(row['fips']): 
        if row['area_name'] in dem_winners_fips_missing:
            democratic_data.loc[index, 'democratic_winner'] = 'Bernie Sanders'
    
    # otherwise look in the winner dict to see who was the winner
    elif row['fips'] in dem_winners:
        democratic_data.loc[index, 'democratic_winner'] = dem_winners[row['fips']][0]
    
    
# remove all the rows where the y variable is missing : value 0

# this means that also the tie rows are dropped:
democratic_data_clean = democratic_data[democratic_data['democratic_winner'].isin \
                                        (['Hillary Clinton', 'Bernie Sanders'])]

# threshold for dropping row due to missing values is 55 (all features) - 1 (y variable) = 54
democratic_data_clean = democratic_data_clean.dropna(axis=0, thresh = 54)

C:\development\anaconda\lib\site-packages\pandas\core\indexes\api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [10]:
# create train/test split 

from sklearn.utils import shuffle

# states for the test split where elections where after april 19 (makes about 30% of total population and 22% of rows): 
# Indiana, West Virginia, Oregon, California, Montana, New Jersey, New Mexico, North Dakota, South Dakota,
# Pennsylvania, Rhode Island, Connecticut, Delaware, Kentucky, and Maryland. 
# (See drive for ordering by date) 
# NB: nebraska is in the republican cutoff and not the democratic. And Kentucky not in the Republican 
# but is in the democratic.

democratic_data_test = democratic_data_clean[democratic_data_clean['state_abbreviation'].isin \
                                (['IN','WV','OR','CA','MT','NJ','NM','ND','SD','PA','RI','CT','DE','MD','KY'])]

democratic_data_train = democratic_data_clean[~ democratic_data_clean['state_abbreviation'].isin \
                                (['IN','WV','OR','CA','MT','NJ','NM','ND','SD','PA','RI','CT','DE','MD','KY'])]

###create X and y for train and test data. 

# now that we do not need the fips and state abbriviations any more, only keep 
# the numeric values and remove 'fips'.
X_train_dem = democratic_data_train._get_numeric_data()

X_train_dem.drop(['fips'], axis=1, inplace=True)

y_train_dem = democratic_data_train['democratic_winner']


# same for test data 
X_TEST_dem = democratic_data_test._get_numeric_data()

X_TEST_dem.drop(['fips'], axis=1, inplace=True)

y_TEST_dem = democratic_data_test['democratic_winner']


# now shuffle them so that future partitioning is certainly random: seed = 123
X_train_dem, y_train_dem = shuffle(X_train_dem, y_train_dem, random_state = 123)

X_TEST_dem, y_TEST_dem = shuffle(X_TEST_dem, y_TEST_dem, random_state = 123)

# Republican train / test split 

In [11]:
# create a dictionary with fips (unique country code) as key and as values and loop over 
# the result data for each candidate. In the first loop the percentage of votes for 
# the first canditate will fill all values of the dict. But in the following loops
# the value will be upgraded by if the other candidate had a higher percentage of the vote 


# 'John Kasich', 'Rand Paul', 'Donald Trump', 'Carly Fiorina', 'Rick Santorum', 'Ted Cruz', 'Jeb Bush', 
# 'Mike Huckabee', 'Ben Carson', 'Marco Rubio', 'Chris Christie'}


rep_winners = {}
rep_winners_fips_missing = {}


for index, row in republican_results.iterrows():
    
    if pd.isnull(row['fips']) and row['candidate'] == 'Donald Trump':
        
        rep_winners_fips_missing[row['county']] = [row['state_abbreviation'], row['fraction_votes'],
                                                   'Donald Trump']
    
    elif row['candidate'] == 'Donald Trump':
        rep_winners[row['fips']] = ['Donald Trump', row['fraction_votes']]
        

# now check for the counties where fips are missing, who are all in new Hampshire, if 
# an other candite scored higher then Trump

for index, row in republican_results.iterrows():
    
    if pd.isnull(row['fips']):
    
        if row['candidate'] == 'Rand Paul' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Rand Paul']

        elif row['candidate'] == 'John Kasich' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'John Kasich']

        elif row['candidate'] == 'Carly Fiorina' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Carly Fiorina']

        elif row['candidate'] == 'Rick Santorum' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Rick Santorum']

        elif row['candidate'] == 'Ted Cruz' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Ted Cruz']

        elif row['candidate'] == 'Jeb Bush' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Jeb Bush']
            
        elif row['candidate'] == 'Mike Huckabee' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Mike Huckabee']

        elif row['candidate'] == 'Ben Carson'and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Ben Carson']
                                                                                                  
        elif row['candidate'] == 'Marco Rubio' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Marco Rubio']

        elif row['candidate'] == 'Chris Christie' and row['fraction_votes'] > rep_winners_fips_missing[row['county']][1]:
            rep_winners_fips_missing[row['county']] = [row['state_abbreviation'],row['fraction_votes'], 'Chris Christie']

# now check for all the rows where fips are present:
for index, row in republican_results.iterrows():
    
    # skip rows where the fip is missing, already did these above.
    if pd.isnull(row['fips']):
        continue 
    
    elif row['candidate'] == 'Rand Paul' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Rand Paul', row['fraction_votes']]
    
    elif row['candidate'] == 'John Kasich' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['John Kasich', row['fraction_votes']]
    
    elif row['candidate'] == 'Carly Fiorina' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Carly Fiorina', row['fraction_votes']]
        
    elif row['candidate'] == 'Rick Santorum' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Rick Santorum', row['fraction_votes']]
        
    elif row['candidate'] == 'Ted Cruz' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Ted Cruz', row['fraction_votes']]
        
    elif row['candidate'] == 'Jeb Bush' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Jeb Bush', row['fraction_votes']]
        
    elif row['candidate'] == 'Mike Huckabee' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Mike Huckabee', row['fraction_votes']]
        
    elif row['candidate'] == 'Ben Carson'and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Ben Carson', row['fraction_votes']]
    
    elif row['candidate'] == 'Marco Rubio' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Marco Rubio', row['fraction_votes']]
    
    elif row['candidate'] == 'Chris Christie' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Chris Christie', row['fraction_votes']]
    

In [12]:
# now check for all the rows where fips are present:

for index, row in republican_results.iterrows():
    
    # skip rows where the fip is missing, already did these above.
    if pd.isnull(row['fips']):
        continue 
    
    elif row['candidate'] == 'Rand Paul' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Rand Paul', row['fraction_votes']]
    
    elif row['candidate'] == 'John Kasich' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['John Kasich', row['fraction_votes']]
    
    elif row['candidate'] == 'Carly Fiorina' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Carly Fiorina', row['fraction_votes']]
        
    elif row['candidate'] == 'Rick Santorum' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Rick Santorum', row['fraction_votes']]
        
    elif row['candidate'] == 'Ted Cruz' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Ted Cruz', row['fraction_votes']]
        
    elif row['candidate'] == 'Jeb Bush' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Jeb Bush', row['fraction_votes']]
        
    elif row['candidate'] == 'Mike Huckabee' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Mike Huckabee', row['fraction_votes']]
        
    elif row['candidate'] == 'Ben Carson'and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Ben Carson', row['fraction_votes']]
    
    elif row['candidate'] == 'Marco Rubio' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Marco Rubio', row['fraction_votes']]
    
    elif row['candidate'] == 'Chris Christie' and row['fraction_votes'] > rep_winners[row['fips']][1]:
        rep_winners[row['fips']] = ['Chris Christie', row['fraction_votes']]

In [13]:
# create the y column: 'republican_winner'. Set values to 0 for now. Merge with the county data.

republican_winner = {'republican_winner' : [0] * len(county_data['fips'])}

republican_winner_df = pd.DataFrame(data = republican_winner)

republican_data = pd.concat([county_data ,republican_winner_df], axis = 1)

C:\development\anaconda\lib\site-packages\pandas\core\indexes\api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [14]:
# Now set the values of the 'republican_winner' column to winner. Value stays 0 if 
# the county code is not in the result data-set, and therefore missing

for index, row in republican_data.iterrows():
    
    # this is the if statement for where country fip is missing and berny is thus the winner. 
    if pd.isnull(row['fips']): 
        if row['area_name'] in rep_winners_fips_missing:
            republican_data.loc[index, 'republican_winner'] = 'Donald Trump'
    
    # otherwise look in the winner dict to see who was the winner
    elif row['fips'] in rep_winners:
        republican_data.loc[index, 'republican_winner'] = rep_winners[row['fips']][0]
    

# remove all the rows where the y variable is missing : value 0
republican_data_clean = republican_data[republican_data['republican_winner'].isin \
                                        (['John Kasich', 'Rand Paul', 'Donald Trump', 
                                        'Carly Fiorina', 'Rick Santorum', 'Ted Cruz', 'Jeb Bush', 
                                        'Mike Huckabee', 'Ben Carson', 'Marco Rubio', 'Chris Christie'])]

In [15]:
# create train/ test split 

republican_data_test = republican_data_clean[republican_data_clean['state_abbreviation'].isin \
                                (['IN','WV','OR','CA','MT','NJ','NM','ND','SD','PA','RI','CT','DE','MD','NE'])]

republican_data_train = republican_data_clean[~ republican_data_clean['state_abbreviation'].isin \
                                (['IN','WV','OR','CA','MT','NJ','NM','ND','SD','PA','RI','CT','DE','MD','NE'])]

######
###### create X and y for train and test data. 
######

# now that we do not need the fips and state abbriviations any more, only keep 
# the numeric values and remove 'fips'.
X_train_rep = republican_data_train._get_numeric_data()

X_train_rep.drop(['fips'], axis=1, inplace=True)

y_train_rep = republican_data_train['republican_winner']


# same for test data 
X_TEST_rep = republican_data_test._get_numeric_data()

X_TEST_rep.drop(['fips'], axis=1, inplace=True)

y_TEST_rep = republican_data_test['republican_winner']


# now shuffle them so that future partitioning is certainly random: seed = 123
X_train_rep, y_train_rep = shuffle(X_train_rep, y_train_rep, random_state = 123)

X_TEST_rep, y_TEST_rep = shuffle(X_TEST_rep, y_TEST_rep, random_state = 123)

# Most_Votes train/test split

In [16]:
# Create dictionary with fips (unique county codes) as keys and another dict as values.
# this second dict contains party as keys and their votes in a list as values.

# Also create the same dict for counties which do not have a fib.

most_votes_dict = {}

most_votes_dict_fips_missing = {}

# first for rows where fips are present:
for index, row in results_data.iterrows():
    
    if not pd.isnull(row['fips']):
        
        if row['fips'] not in most_votes_dict:
            
            most_votes_dict[row['fips']] = {'Democrats': 0, 'Republicans': 0}

            
for index, row in results_data.iterrows():
    
    if not pd.isnull(row['fips']) and row['fips'] in most_votes_dict:
            
            if row['party'] == 'Democrat':
                most_votes_dict[row['fips']]['Democrats'] += row['votes']
                
            elif row['party'] == 'Republican':
                most_votes_dict[row['fips']]['Republicans'] += row['votes']


                
# now for the rows where fip is missing:
for index, row in results_data.iterrows():    
    
    if pd.isnull(row['fips']):
        
        if row['county'] not in most_votes_dict_fips_missing:
            most_votes_dict_fips_missing[row['county']] = {'Democrats': 0, 'Republicans': 0}

for index, row in results_data.iterrows():    
    
    if pd.isnull(row['fips']) and row['county'] in most_votes_dict_fips_missing:
            
        if row['party'] == 'Democrat':
            most_votes_dict_fips_missing[row['county']]['Democrats'] += row['votes']
                
        elif row['party'] == 'Republican':
            most_votes_dict_fips_missing[row['county']]['Republicans'] += row['votes']
            

most_votes = {'most_votes' : [0] * len(county_data['fips'])}

most_votes_df = pd.DataFrame(data = most_votes)

votes_data = pd.concat([county_data ,most_votes_df], axis = 1)

# now set the y values to the party with the most votes. Value stays 0 if 
# the county code is not in the result data-set, and therefore missing

for index, row in votes_data.iterrows():
    
    # this is the if statement for where country fip is missing and berny is thus the winner. 
    if pd.isnull(row['fips']): 
        
        if row['area_name'] in most_votes_dict_fips_missing:
            
            winner = None 
            
            if most_votes_dict_fips_missing[row['area_name']]['Republicans'] > \
                most_votes_dict_fips_missing[row['area_name']]['Democrats']:
                winner = 'Republicans'
            
            elif most_votes_dict_fips_missing[row['area_name']]['Democrats'] > \
                most_votes_dict_fips_missing[row['area_name']]['Republicans']:
                winner = 'Democrats'
            
            votes_data.loc[index, 'most_votes'] = winner 
    
    # otherwise look in the winner dict to see who was the winner
    elif row['fips'] in most_votes_dict:
        
            winner = None 
            
            if most_votes_dict[row['fips']]['Republicans'] > most_votes_dict[row['fips']]['Democrats']:
                winner = 'Republicans'
            
            elif most_votes_dict[row['fips']]['Democrats'] > most_votes_dict[row['fips']]['Republicans']:
                winner = 'Democrats'
    
            votes_data.loc[index, 'most_votes'] = winner

        
    
# remove all the rows where the y variable is missing : value 0
votes_data_clean = votes_data[votes_data['most_votes'].isin \
                                        (['Republicans', 'Democrats'])]


C:\development\anaconda\lib\site-packages\pandas\core\indexes\api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [17]:
# create train/test split 

votes_data_test = votes_data_clean[votes_data_clean['state_abbreviation'].isin \
                                (['IN','WV','OR','CA','MT','NJ','NM','ND','SD','PA','RI','CT','DE','MD','KY'])]

votes_data_train = votes_data_clean[~ votes_data_clean['state_abbreviation'].isin \
                                (['IN','WV','OR','CA','MT','NJ','NM','ND','SD','PA','RI','CT','DE','MD','KY'])]

######
###### create X and y for train and test data. 
######

# now that we do not need the fips and state abbriviations any more, only keep 
# the numeric values and remove 'fips'.
X_train_votes = votes_data_train._get_numeric_data()

X_train_votes.drop(['fips'], axis=1, inplace=True)

y_train_votes = votes_data_train['most_votes']


# same for test data 
X_TEST_votes = votes_data_test._get_numeric_data()

X_TEST_votes.drop(['fips'], axis=1, inplace=True)

y_TEST_votes = votes_data_test['most_votes']


# now shuffle them so that future partitioning is certainly random: seed = 123
X_train_votes, y_train_votes = shuffle(X_train_votes, y_train_votes, random_state = 123)

X_TEST_votes, y_TEST_votes = shuffle(X_TEST_votes, y_TEST_votes, random_state = 123)

# New Models: 

### Most Votes

In [18]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

logreg_pipe1 = make_pipeline(PolynomialFeatures(),MinMaxScaler(), LogisticRegression(random_state=123))
param_grid1 = {'logisticregression__C': np.logspace(-6,3,14)}
grid1 = GridSearchCV(logreg_pipe1, param_grid1, cv=5)
grid1.fit(X_train_votes, y_train_votes)
print("The best performance: {:.3f}".format(grid1.score(X_train_votes, y_train_votes)))
print("The corresponding regularization parameter C is: {:.3f}".format(grid1.best_params_['logisticregression__C']))

The best performance: 0.923
The corresponding regularization parameter C is: 8.377


In [19]:
print(grid1.score(X_TEST_votes, y_TEST_votes))

0.651347068146


### Democrats 

In [20]:
logreg_pipe2 = make_pipeline(PolynomialFeatures(),MinMaxScaler(), LogisticRegression(random_state=123))
param_grid2 = {'logisticregression__C': np.logspace(-6,3,14)}
grid2 = GridSearchCV(logreg_pipe2, param_grid2, cv=5)
grid2.fit(X_train_dem, y_train_dem)
print("The best performance: {:.3f}".format(grid2.score(X_train_dem, y_train_dem)))
print("The corresponding regularization parameter C is: {:.3f}".format(grid2.best_params_['logisticregression__C']))

The best performance: 0.903
The corresponding regularization parameter C is: 41.246


In [21]:
print(grid2.score(X_TEST_dem, y_TEST_dem))

0.522364217252


### Republicans 

In [22]:
logreg_pipe3 = make_pipeline(PolynomialFeatures(),MinMaxScaler(), LogisticRegression(random_state=123))
param_grid3 = {'logisticregression__C': np.logspace(-6,3,14)}
grid3 = GridSearchCV(logreg_pipe3, param_grid3, cv=5)
grid3.fit(X_train_rep, y_train_rep)
print("The best performance: {:.3f}".format(grid3.score(X_train_rep, y_train_rep)))
print("The corresponding regularization parameter C is: {:.3f}".format(grid3.best_params_['logisticregression__C']))

The best performance: 0.867
The corresponding regularization parameter C is: 8.377


In [23]:
print(grid3.score(X_TEST_rep, y_TEST_rep))

0.699834162521
